In [ ]:
!pip install wikipedia-api
!pip install chromadb
!pip install langchain
!pip install langchain-openai
!pip install langchain_community

In [2]:
import os

os.environ['OPENAI_API_KEY'] = '<OPENAI_API_KEY>'

In [3]:
import wikipediaapi

language_code = 'en'

wiki = wikipediaapi.Wikipedia(
  user_agent='MyProjectName (merlin@example.com)',
  language=language_code,
  extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [4]:
topic = 'world chess champion'
query = ''
pages = 5

In [5]:
import requests
import json

headers = {
  # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
  'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
}

base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
endpoint = '/search/page'
url = base_url + language_code + endpoint
parameters = {'q': topic, 'limit': pages}
response = requests.get(url, headers=headers, params=parameters)
response = json.loads(response.text)

titles = []
for page in response['pages']:
  titles.append(page['title'])

texts = []
for title in titles:
  page = wiki.page(title)
  print(title)
  text = page.text
  texts.append(text)

World Chess Championship
Women's World Chess Championship
List of World Chess Championships
World Chess Championship 2023
Tata Steel Chess Tournament 2024


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.schema.document import Document

text_splitter = CharacterTextSplitter(
  chunk_size=100,
  separator="\n",
  chunk_overlap=0)

chunks = []
for text in texts:
  text2 = text.replace(".", "\n")
  text2 = text2.replace(";", "\n")
  for chunk in text_splitter.split_text(text2):
    chunks.append(chunk)
    #print(chunk)
    #print('-------------------------------------------------------------------------------------------------------------------------------------------------------')

chunks = list(dict.fromkeys(chunks))

docs = []
for c in chunks:
  doc = Document(page_content=c)
  docs.append(doc)

print('-------------------------------------------------------------------------------------------------------------------------------------------------------')
print('chunks:', len(docs))

chroma = Chroma()
chroma.delete_collection()

db = chroma.from_documents(docs, OpenAIEmbeddings())
#result = db.similarity_search(query, k=10)

#print('-------------------------------------------------------------------------------------------------------------------------------------------------------')
#for x in result:
  #print(x.page_content)


In [7]:
print('chunks:', len(docs))

chunks: 944


In [8]:
import os
from openai import OpenAI

def generate(contexts):
  client = OpenAI()
  OpenAI.api_key = os.getenv('OPENAI_API_KEY')

  context_str = "\n".join(contexts)

  prompt = f"""You are a helpful assistant, below is a query from a user and
  some relevant contexts. Answer the question given the information in those
  contexts. If you cannot find the answer to the question, say "I don't know".
  Contexts:
  {context_str}
  Query: {query}
  Answer: """

  completion = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=100,
    temperature=0
  )

  print(query)
  print(completion.choices[0].text)

In [10]:
query = 'who is current world chess champion?'

result = db.similarity_search(query, k=10)

context = [x.page_content for x in result]
generate(context)

print('------------------------------------------------------------------------')
for r in result:
  print(r)

who is current world chess champion?
Ding Liren
------------------------------------------------------------------------
page_content='The current world champion is Ding Liren, who defeated his opponent Ian Nepomniachtchi in the 2023 World Chess Championship'
page_content='This tournament was won by Viswanathan Anand, thus making him the World Chess Champion'
page_content='The World Chess Championship is played to determine the world champion in chess'
page_content='The previous World Champion was Magnus Carlsen, who first won the title in 2013'
page_content='FIDE World Championships (2006–present)'
page_content='World Junior Chess Championship (under 20 years of age)'
page_content='At the time the championship was called the Braingames World Chess Championship, but Kramnik later referred to himself as the Classical World Chess Champion'
page_content='The World Chess Championship 2023 was a chess match between Ian Nepomniachtchi and Ding Liren to determine the new World Chess Champion'

In [11]:
generate([])

who is current world chess champion?

The current world chess champion is Magnus Carlsen.
